In [ ]:
"""
================================================================================
PDF ANNOTATION EXTRACTOR FOR RESEARCH
================================================================================
A simple script to extract highlights and comments from annotated PDFs and 
organize them into a CSV file for qualitative analysis.

WHAT YOU NEED:
- Python 3.7 or higher
- PyMuPDF library (install with: pip install PyMuPDF)
- Mendeley or any PDF reader that supports highlights and comments
- Your research articles in PDF format
- A CSV file with your article metadata (must include a DOI column)

WORKFLOW:
1. Annotate PDFs in Mendeley (or any PDF reader)
2. Add a comment "DOI: [your-doi-here]" to the first page
3. Highlight text and add comments like "Theme>Subtheme>Detail"
4. Run this script
5. Open the output CSV in Excel to see your coded data

Author: [Your Name]
License: MIT (free to use and modify)
================================================================================
"""

import fitz  # PyMuPDF
import pandas as pd

# ============================================================================
# CONFIGURATION SECTION - MODIFY THESE FOR YOUR SETUP
# ============================================================================

# Define your highlight color categories
# NOTE: These RGB values might differ based on your PDF reader
# To find your actual colors, run the diagnostic script first (see below)
COLOR_MAP = {
    "1.0,0.9,0.4": "RQ",                # Yellow highlights
    "1.0,0.6,0.8": "Result",            # Pink highlights
    "0.0,1.0,0.0": "Theory",            # Green highlights
    "0.0,1.0,1.0": "Theme",             # Cyan highlights
    "1.0,0.0,0.0": "Future Research"    # Red highlights
}

# Set your file paths here
# TIP: Use raw strings (r"...") to avoid issues with Windows backslashes
csv_path = r"C:\Users\YourName\Documents\research\articles.csv"
pdf_path = r"C:\Users\YourName\Documents\research\smith-2024.pdf"

# ============================================================================
# MAIN EXTRACTION LOGIC - YOU PROBABLY DON'T NEED TO MODIFY THIS
# ============================================================================

print("=" * 70)
print("PDF ANNOTATION EXTRACTOR")
print("=" * 70)

# Step 1: Load your master CSV file
print("\n📂 Loading CSV file...")
df = pd.read_csv(csv_path)
df['DOI'] = df['DOI'].astype(str).str.strip().str.lower()
print(f"✓ Loaded {len(df)} articles from CSV")

# Step 2: Open the PDF file
print(f"\n📖 Opening PDF file...")
doc = fitz.open(pdf_path)
print(f"✓ PDF opened successfully ({len(doc)} pages)")

# Step 3: Find the DOI from the PDF annotations
print("\n🔍 Looking for DOI in PDF annotations...")
pdf_doi = None

for page in doc:
    for annot in page.annots():
        content = annot.info.get("content", "").strip()
        # Look for a comment that starts with "DOI:"
        if content.lower().startswith("doi:"):
            pdf_doi = content.split(":", 1)[1].strip().lower()
            print(f"✓ Found DOI: {pdf_doi}")
            break
    if pdf_doi:
        break

if not pdf_doi:
    print("❌ ERROR: No DOI found in PDF annotations!")
    print("   Make sure you added a comment 'DOI: [your-doi]' to the first page")
    exit()

# Step 4: Find the matching row in your CSV
print(f"\n🔎 Searching for DOI in CSV...")
match = df[df['DOI'] == pdf_doi]

if match.empty:
    print(f"❌ ERROR: DOI '{pdf_doi}' not found in CSV")
    print("   Check that the DOI matches exactly (case-insensitive)")
    print(f"   First few DOIs in your CSV: {df['DOI'].head(3).tolist()}")
    exit()

row_idx = match.index[0]
print(f"✓ Match found at row {row_idx}")

# Step 5: Extract all annotations from the PDF
print(f"\n📝 Extracting annotations...")
codes = {}  # Dictionary to store: {column_name: [list of highlighted texts]}

for page_num, page in enumerate(doc, 1):
    for annot in page.annots():
        content = annot.info.get("content", "").strip()
        color = ",".join(f"{v:.1f}" for v in annot.colors.get("stroke", (0,0,0)))
        
        # Skip the DOI annotation
        if content.lower().startswith("doi:"):
            continue
        
        # Skip if this color is not in our COLOR_MAP
        if color not in COLOR_MAP:
            continue
        
        # Extract the highlighted text from the PDF
        highlight_text = ""
        if annot.vertices:
            quads = [fitz.Quad(annot.vertices[i:i+4]) for i in range(0, len(annot.vertices), 4)]
            for quad in quads:
                highlight_text += page.get_textbox(quad.rect) + " "
        highlight_text = highlight_text.strip()
        
        # Skip if no text was highlighted
        if not highlight_text:
            continue
        
        # Parse the comment to create column name
        # Example: "Theory>Antecedent>Signaling" becomes "Theory_Antecedent_Signaling"
        if content:
            # Split by ">" to create hierarchy
            levels = [level.strip() for level in content.split(">")]
            col_name = "_".join(levels)
        else:
            # If no comment, just use the color category
            col_name = COLOR_MAP[color]
        
        # Store the highlighted text
        if col_name not in codes:
            codes[col_name] = []
        codes[col_name].append(highlight_text)
        
        print(f"  Page {page_num}: {col_name[:50]}...")

print(f"\n✓ Extracted {sum(len(v) for v in codes.values())} annotations across {len(codes)} unique codes")

# Step 6: Write annotations to the CSV
print(f"\n💾 Writing annotations to CSV...")

for col_name, texts in codes.items():
    # Add the column if it doesn't exist yet
    if col_name not in df.columns:
        df[col_name] = ""
    
    # Join multiple highlights with " | " separator
    df.at[row_idx, col_name] = " | ".join(texts)
    print(f"  ✓ {col_name}: {len(texts)} quote(s)")

# Step 7: Save the updated CSV
output_path = csv_path.replace('.csv', '_annotated.csv')
df.to_csv(output_path, index=False)

print(f"\n" + "=" * 70)
print(f"✅ SUCCESS!")
print(f"=" * 70)
print(f"Output saved to: {output_path}")
print(f"\nYou can now open this file in Excel to see your coded data.")
print(f"Each unique code path (e.g., 'Theory_Antecedent_Signaling') appears as a column.")
print(f"All highlighted text for that code is in the corresponding cell.")
print("=" * 70)